In [154]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


#scrapping tools etc from isabella-b.com link
from requests import get
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')


#lec 17
import csv
import itertools
import operator
import sys
from datetime import datetime

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [155]:
"""
Peter Mansbacher
Manthan Baboo
CSCI 4931
Professor Ashis Biswas
Final Project : Naruto Shippuden new episode text generator

Citations
Lecture 17
https://dev.to/magesh236/scrape-imdb-movie-rating-and-details-3a7c
https://isabella-b.com/blog/scraping-episode-imdb-ratings-tutorial/
https://www.nltk.org/howto/stem.html
"""

'\nPeter Mansbacher\nManthan Baboo\nCSCI 4931\nProfessor Ashis Biswas\nFinal Project :\n\nCitations\nLecture 17\nhttps://dev.to/magesh236/scrape-imdb-movie-rating-and-details-3a7c\nhttps://isabella-b.com/blog/scraping-episode-imdb-ratings-tutorial/\nhttps://www.nltk.org/howto/stem.html\n'

In [156]:
the_url="https://www.imdb.com/title/tt0988824/"

r = requests.get(url=the_url)
#create a beautiful soup object
soup = BeautifulSoup(r.text,'html.parser')
data = {}

In [157]:
#page title
title=soup.find('title')
print(title.string)
data["title"]=title.string

Naruto: Shippûden (TV Series 2007–2017) - IMDb


In [158]:
#first season season1,ep1 here
url_ = "https://www.imdb.com/title/tt0988824/episodes?year=2009"
response = get(url_)

In [159]:
type(soup)

bs4.BeautifulSoup

In [160]:
naruto_episodes=[]

response = get(url_)
page_html= BeautifulSoup(response.text, 'html.parser')
episode_containers=page_html.find_all('div',class_='info')
sn_ = 1

#2009
for episodes in episode_containers:
    season=sn_
    episode_number=episodes.meta['content']
    title=episodes.a['title']
    desc = episodes.find('div', class_='item_description').text.strip()
    # Compiling the episode info
    episode_data = [season, episode_number, title, desc]

    # Append the episode info to the complete dataset
    naruto_episodes.append(episode_data)

    
#2010-2017    
for sn in range(10,18):
    
    response = get("https://www.imdb.com/title/tt0988824/episodes?year=20" + str(sn))
    page_html= BeautifulSoup(response.text, 'html.parser')
    episode_containers=page_html.find_all('div',class_='info')
    #print("https://www.imdb.com/title/tt0988824/episodes?year=20" + str(sn))
    for episodes in episode_containers:
        #print("Season {}".format(sn))
        season=sn
        episode_number=episodes.meta['content']
        #print(episode_number)
        title=episodes.a['title']
        desc = episodes.find('div', class_='item_description').text.strip()
        # Compiling the episode info
        episode_data = [season, episode_number, title, desc]

        # Append the episode info to the complete dataset
        naruto_episodes.append(episode_data)



    
print("Pages scrapped")    

Pages scrapped


In [161]:
naruto_episodes= pd.DataFrame(naruto_episodes, columns = ['season', 'episode_number', 'title', 'desc'])

naruto_episodes.tail()


,season,episode_number,title,desc
497,17,496,Yukemuri to hyôrôgan,The game of finding gift for naruto's wedding ...
498,17,497,Kazekage no oiwai,The Tsuchikage and Raikage come to Leaf Villag...
499,17,498,Saigo no ninmu,Kiba makes it his goal to become the next Hoka...
500,17,499,Gokuhi ninmu no yukue,"As the wedding day approaches, each of Naruto'..."
501,17,500,Iwai no kotoba,"The day of the wedding finally arrives, as iru..."


In [162]:
naruto_episodes.to_csv('Naruto_Episodes_IMDB.csv',index=True)


In [163]:
temp_ =naruto_episodes[['desc']]
temp_.head()
temp_.to_csv('Naruto_Episodes_IMDB_Descriptions.csv',index=False)

In [164]:
data = pd.read_csv('Naruto_Episodes_IMDB.csv')
df = data[['title','desc']]
df['Index'] = np.arange(0,502)
df.head()

,title,desc,Index
0,Kikyô,Naruto returns to Konoha after a two-and-a-hal...,0
1,"Akatsuki, shidô",Kakashi gives Naruto and Sakura a test to see ...,1
2,Shûgyô no seika,Sakura and Naruto both demonstrate their newly...,2
3,Suna no jinchûriki,"Gaara is quick to detect Deidara's presence, a...",3
4,Kazekage to shite...!,Naruto and Sakura defeat Kakashi in the surviv...,4


In [165]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()
import re
summary = []
for i in range(0,len(df)):
    about = re.sub('[^a-zA-Z]', ' ', df['desc'][i])
    about = about.lower()
    about = about.split()
    about = [stemmer.stem(word) for word in about if not word in stop_words]
    about = ' '.join(about)
    summary.append(about)

In [166]:
summary

['naruto return konoha two half year train journey jiraiya reunit sakura',
 'kakashi give naruto sakura test see far improv meanwhil two akatsuki member deidara sasori arriv entranc sunagakur captur one tail shukaku host gaara',
 'sakura naruto demonstr newli acquir skill meanwhil deidara commenc attack sunagakur confront gaara',
 'gaara quick detect deidara presenc aerial battl ensu deidara giant bird gaara atop sand',
 'naruto sakura defeat kakashi surviv challeng tsunad assign three man squad led kakashi tell equal',
 'order protect villag deidara explos clay gaara use chakra creat giant sand shield',
 'gaara strength complet exhaust whisk away deidara kankuro ignor baki effort stop immedi set pursuit platoon rescu gaara',
 'way sand villag temari encount kakashi team kakashi admonish naruto tri rush ahead',
 'urgent messag sand villag detail akatsuki kidnap kazekag reach hidden leaf villag tsunad wast time order team kakashi sand villag',
 'sand ninja fight time treat kankuro even 

In [233]:
vocabulary_size =8000
unknown_token = "UNKNOWN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [234]:
with open('Naruto_Episodes_IMDB_Descriptions.csv', 'rt') as f:
    reader = csv.reader(f, skipinitialspace=True)
    #reader.next()
    next(reader)
    
    # Split full comments into sentences, and chain with the previous sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print( "Parsed %d sentences." % (len(sentences)))

Parsed 1113 sentences.


In [235]:
sentences

['SENTENCE_START naruto returns to konoha after a two-and-a-half-year training journey with jiraiya and is reunited with sakura. SENTENCE_END',
 'SENTENCE_START kakashi gives naruto and sakura a test to see how far they had improved. SENTENCE_END',
 "SENTENCE_START meanwhile, two akatsuki members, deidara and sasori, arrive at the entrance of the sunagakure to capture the one-tailed shukaku's host, gaara. SENTENCE_END",
 'SENTENCE_START sakura and naruto both demonstrate their newly acquired skills. SENTENCE_END',
 'SENTENCE_START meanwhile, deidara commences his attack on sunagakure and is confronted by gaara. SENTENCE_END',
 "SENTENCE_START gaara is quick to detect deidara's presence, and an aerial battle ensues with deidara on his giant bird and gaara atop sand. SENTENCE_END",
 'SENTENCE_START naruto and sakura defeat kakashi in the survival challenge. SENTENCE_END',
 'SENTENCE_START tsunade assigns them to a three-man squad led by kakashi, who tells them they are now his equals. SE

In [236]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [237]:
tokenized_sentences

[['SENTENCE_START',
  'naruto',
  'returns',
  'to',
  'konoha',
  'after',
  'a',
  'two-and-a-half-year',
  'training',
  'journey',
  'with',
  'jiraiya',
  'and',
  'is',
  'reunited',
  'with',
  'sakura',
  '.',
  'SENTENCE_END'],
 ['SENTENCE_START',
  'kakashi',
  'gives',
  'naruto',
  'and',
  'sakura',
  'a',
  'test',
  'to',
  'see',
  'how',
  'far',
  'they',
  'had',
  'improved',
  '.',
  'SENTENCE_END'],
 ['SENTENCE_START',
  'meanwhile',
  ',',
  'two',
  'akatsuki',
  'members',
  ',',
  'deidara',
  'and',
  'sasori',
  ',',
  'arrive',
  'at',
  'the',
  'entrance',
  'of',
  'the',
  'sunagakure',
  'to',
  'capture',
  'the',
  'one-tailed',
  'shukaku',
  "'s",
  'host',
  ',',
  'gaara',
  '.',
  'SENTENCE_END'],
 ['SENTENCE_START',
  'sakura',
  'and',
  'naruto',
  'both',
  'demonstrate',
  'their',
  'newly',
  'acquired',
  'skills',
  '.',
  'SENTENCE_END'],
 ['SENTENCE_START',
  'meanwhile',
  ',',
  'deidara',
  'commences',
  'his',
  'attack',
  'on',

In [238]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 3119 unique words tokens.


In [239]:
word_freq

FreqDist({'the': 1276, 'SENTENCE_START': 1113, 'SENTENCE_END': 1113, '.': 1023, ',': 850, 'to': 742, 'and': 633, 'of': 360, 'a': 353, 'naruto': 325, ...})

In [240]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])


In [241]:
word_to_index

{'the': 0,
 'SENTENCE_START': 1,
 'SENTENCE_END': 2,
 '.': 3,
 ',': 4,
 'to': 5,
 'and': 6,
 'of': 7,
 'a': 8,
 'naruto': 9,
 'his': 10,
 "'s": 11,
 'is': 12,
 'in': 13,
 'with': 14,
 'that': 15,
 'sasuke': 16,
 'he': 17,
 'as': 18,
 'kakashi': 19,
 'but': 20,
 'for': 21,
 'their': 22,
 'by': 23,
 'village': 24,
 'on': 25,
 'from': 26,
 'team': 27,
 'who': 28,
 'leaf': 29,
 'him': 30,
 'ninja': 31,
 'sakura': 32,
 'out': 33,
 'they': 34,
 'an': 35,
 'are': 36,
 'at': 37,
 'battle': 38,
 'her': 39,
 'hidden': 40,
 'itachi': 41,
 'has': 42,
 'after': 43,
 'shinobi': 44,
 'into': 45,
 'tsunade': 46,
 'jutsu': 47,
 'madara': 48,
 'akatsuki': 49,
 'when': 50,
 'up': 51,
 'meanwhile': 52,
 'be': 53,
 'sai': 54,
 'tails': 55,
 '...': 56,
 'against': 57,
 'about': 58,
 'it': 59,
 'them': 60,
 'back': 61,
 'hokage': 62,
 'asuma': 63,
 'gaara': 64,
 'orochimaru': 65,
 'jiraiya': 66,
 'shikamaru': 67,
 'this': 68,
 'training': 69,
 'time': 70,
 'she': 71,
 'attack': 72,
 'begins': 73,
 'was': 74,

In [242]:
print("Using vocabulary size %d." % vocabulary_size)
print()

print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
word_freq.most_common(10)

Using vocabulary size 4000.

The least frequent word in our vocabulary is 'figures' and appeared 1 times.


[('the', 1276),
 ('SENTENCE_START', 1113),
 ('SENTENCE_END', 1113),
 ('.', 1023),
 (',', 850),
 ('to', 742),
 ('and', 633),
 ('of', 360),
 ('a', 353),
 ('naruto', 325)]

In [243]:
vocab

[('the', 1276),
 ('SENTENCE_START', 1113),
 ('SENTENCE_END', 1113),
 ('.', 1023),
 (',', 850),
 ('to', 742),
 ('and', 633),
 ('of', 360),
 ('a', 353),
 ('naruto', 325),
 ('his', 306),
 ("'s", 305),
 ('is', 260),
 ('in', 196),
 ('with', 195),
 ('that', 153),
 ('sasuke', 146),
 ('he', 132),
 ('as', 128),
 ('kakashi', 126),
 ('but', 125),
 ('for', 119),
 ('their', 118),
 ('by', 109),
 ('village', 109),
 ('on', 105),
 ('from', 99),
 ('team', 99),
 ('who', 88),
 ('leaf', 88),
 ('him', 85),
 ('ninja', 85),
 ('sakura', 84),
 ('out', 76),
 ('they', 75),
 ('an', 74),
 ('are', 74),
 ('at', 73),
 ('battle', 68),
 ('her', 64),
 ('hidden', 62),
 ('itachi', 62),
 ('has', 61),
 ('after', 57),
 ('shinobi', 55),
 ('into', 54),
 ('tsunade', 52),
 ('jutsu', 52),
 ('madara', 51),
 ('akatsuki', 48),
 ('when', 48),
 ('up', 47),
 ('meanwhile', 45),
 ('be', 45),
 ('sai', 44),
 ('tails', 43),
 ('...', 42),
 ('against', 42),
 ('about', 42),
 ('it', 40),
 ('them', 39),
 ('back', 38),
 ('hokage', 38),
 ('asuma', 

In [244]:
index_to_word

['the',
 'SENTENCE_START',
 'SENTENCE_END',
 '.',
 ',',
 'to',
 'and',
 'of',
 'a',
 'naruto',
 'his',
 "'s",
 'is',
 'in',
 'with',
 'that',
 'sasuke',
 'he',
 'as',
 'kakashi',
 'but',
 'for',
 'their',
 'by',
 'village',
 'on',
 'from',
 'team',
 'who',
 'leaf',
 'him',
 'ninja',
 'sakura',
 'out',
 'they',
 'an',
 'are',
 'at',
 'battle',
 'her',
 'hidden',
 'itachi',
 'has',
 'after',
 'shinobi',
 'into',
 'tsunade',
 'jutsu',
 'madara',
 'akatsuki',
 'when',
 'up',
 'meanwhile',
 'be',
 'sai',
 'tails',
 '...',
 'against',
 'about',
 'it',
 'them',
 'back',
 'hokage',
 'asuma',
 'gaara',
 'orochimaru',
 'jiraiya',
 'shikamaru',
 'this',
 'training',
 'time',
 'she',
 'attack',
 'begins',
 'was',
 'mission',
 'obito',
 'chakra',
 'fight',
 'all',
 'have',
 'can',
 'new',
 'uchiha',
 'while',
 'two',
 'one',
 '!',
 'power',
 'kabuto',
 "'",
 'deidara',
 'will',
 'chiyo',
 'find',
 'war',
 'over',
 'danzo',
 'yamato',
 'sora',
 'no',
 'pain',
 'beast',
 'finally',
 'sand',
 'help',


In [245]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [246]:
print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])


Example sentence: 'SENTENCE_START naruto returns to konoha after a two-and-a-half-year training journey with jiraiya and is reunited with sakura. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'naruto', 'returns', 'to', 'konoha', 'after', 'a', 'two-and-a-half-year', 'training', 'journey', 'with', 'jiraiya', 'and', 'is', 'reunited', 'with', 'sakura', '.', 'SENTENCE_END']'


In [247]:
# Create the training data
XTrain = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
yTrain = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [248]:
XTrain

array([list([1, 9, 368, 5, 319, 43, 8, 1452, 69, 724, 14, 66, 6, 12, 427, 14, 32, 3]),
       list([1, 19, 428, 9, 6, 32, 8, 369, 5, 293, 113, 725, 34, 217, 1453, 3]),
       list([1, 52, 4, 85, 49, 185, 4, 91, 6, 114, 4, 170, 37, 0, 1454, 7, 0, 726, 5, 294, 0, 1455, 1456, 11, 958, 4, 64, 3]),
       ...,
       list([1, 168, 274, 59, 10, 310, 5, 214, 0, 257, 62, 198, 484, 923, 3110, 58, 10, 191, 310, 6, 320, 33, 14, 168, 6, 572, 25, 22, 142, 75, 18, 27, 3111, 3]),
       list([1, 18, 0, 367, 281, 415, 4, 260, 7, 9, 11, 233, 3112, 8, 723, 4, 20, 100, 86, 3113, 22, 75, 11, 300, 3114, 913, 0, 1451, 92, 53, 3115, 5, 1149, 28, 81, 676, 0, 367, 3116, 198, 19, 73, 5, 3117, 0, 246, 3]),
       list([1, 0, 281, 7, 0, 367, 103, 148, 4, 18, 258, 103, 3118, 33, 0, 669, 433, 3])],
      dtype=object)

In [249]:
yTrain

array([list([9, 368, 5, 319, 43, 8, 1452, 69, 724, 14, 66, 6, 12, 427, 14, 32, 3, 2]),
       list([19, 428, 9, 6, 32, 8, 369, 5, 293, 113, 725, 34, 217, 1453, 3, 2]),
       list([52, 4, 85, 49, 185, 4, 91, 6, 114, 4, 170, 37, 0, 1454, 7, 0, 726, 5, 294, 0, 1455, 1456, 11, 958, 4, 64, 3, 2]),
       ...,
       list([168, 274, 59, 10, 310, 5, 214, 0, 257, 62, 198, 484, 923, 3110, 58, 10, 191, 310, 6, 320, 33, 14, 168, 6, 572, 25, 22, 142, 75, 18, 27, 3111, 3, 2]),
       list([18, 0, 367, 281, 415, 4, 260, 7, 9, 11, 233, 3112, 8, 723, 4, 20, 100, 86, 3113, 22, 75, 11, 300, 3114, 913, 0, 1451, 92, 53, 3115, 5, 1149, 28, 81, 676, 0, 367, 3116, 198, 19, 73, 5, 3117, 0, 246, 3, 2]),
       list([0, 281, 7, 0, 367, 103, 148, 4, 18, 258, 103, 3118, 33, 0, 669, 433, 3, 2])],
      dtype=object)

In [250]:
# Print an training data example
x_example, y_example = XTrain[17], yTrain[17]
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print ("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START the only hope lies with the reinforcements sent by tsunade .
[1, 0, 115, 1469, 972, 14, 0, 583, 734, 23, 46, 3]

y:
the only hope lies with the reinforcements sent by tsunade . SENTENCE_END
[0, 115, 1469, 972, 14, 0, 583, 734, 23, 46, 3, 2]


In [251]:
class RNNVanilla:
     
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        
        # Assign instance variables
        self.word_dim = word_dim   #size of the vocabulary
        self.hidden_dim = hidden_dim  # size of hidden layer
        self.bptt_truncate = bptt_truncate
        
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))


In [252]:

def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

In [253]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
        
    # During forward propagation we save all hidden states in s because need them later.
        
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
        
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
        
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]  #We not only return the calculated outputs, but also the hidden states. 
                   #We will use them later to calculate the gradients

#Now make it a member of the class RNNVanilla
RNNVanilla.forward_propagation = forward_propagation

In [254]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

#Now make it a member of the class RNNVanilla
RNNVanilla.predict = predict

In [255]:
np.random.seed(10)
model = RNNVanilla(vocabulary_size)
o, s = model.forward_propagation(XTrain[10])
print(o.shape)
print(o)
# For each word in the sentence, the model made 8000 predictions representing probabilities of the next word
# Note that because we initialized U,V,W to random values these predictions are completely random right now. 

(22, 4000)
[[0.00025132 0.00024975 0.0002488  ... 0.00025076 0.00024852 0.00025015]
 [0.00025115 0.00024756 0.00025079 ... 0.00025068 0.00024847 0.00024751]
 [0.00024907 0.00025095 0.00024835 ... 0.000251   0.00024883 0.00024833]
 ...
 [0.00025025 0.00024699 0.00025143 ... 0.00025219 0.0002524  0.00025141]
 [0.00025274 0.00025183 0.00025278 ... 0.00024855 0.00024761 0.00025078]
 [0.00025246 0.00024778 0.00024948 ... 0.00025005 0.00024878 0.00024986]]


In [256]:
#The following gives the indices of the highest probability predictions for each word:
predictions = model.predict(XTrain[10])
print(predictions.shape)
print(predictions)
print("index_to_word>")
print('%s'%" ".join([index_to_word[y] for y in range(len(predictions))]))

(22,)
[ 650 3381  309  514  762 1549 2423 2528 2812 3430 3150 2545  783 1039
  110  410 3521 2039  326 1712 3994 1151]
index_to_word>
the SENTENCE_START SENTENCE_END . , to and of a naruto his 's is in with that sasuke he as kakashi but for


In [257]:
def calculate_total_loss(self, x, y):
    L = 0
    
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        
        # Add to the loss based on how off we were
        L += -1 * sum(np.log(correct_word_predictions))
    return L
 
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N
 
RNNVanilla.calculate_total_loss = calculate_total_loss
RNNVanilla.calculate_loss = calculate_loss

In [258]:
#yield total loss of 1/C or log(c)
np.log(vocabulary_size) #= log(C)

8.294049640102028

In [259]:
XTrain[:100].shape

(100,)

In [260]:
XTrain.shape

(1113,)

In [261]:
# Limit to 1000 examples to save time
print("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(XTrain[:1000], yTrain[:1000]))

Expected Loss for random predictions: 8.294050
Actual loss: 8.294093


In [262]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNVanilla.bptt = bptt

In [263]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print ("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print ("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print ("+h Loss: %f" % gradplus)
                print ("-h Loss: %f" % gradminus)
                print ("Estimated_gradient: %f" % estimated_gradient)
                print ("Backpropagation gradient: %f" % backprop_gradient)
                print ("Relative Error: %f" % relative_error)
                return
            it.iternext()
        print ("Gradient check for parameter %s passed." % (pname))
 
RNNVanilla.gradient_check = gradient_check

In [264]:
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNVanilla(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [265]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
    
RNNVanilla.sgd_step = numpy_sdg_step

In [266]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [267]:
np.random.seed(10)
model = RNNVanilla(vocabulary_size)
%timeit model.sgd_step(XTrain[10], yTrain[10], learning_rate=0.005)

133 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [268]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNVanilla(vocabulary_size)
losses = train_with_sgd(model, XTrain[:100], yTrain[:100], nepoch=10, evaluate_loss_after=1)

2021-05-11 09:50:04: Loss after num_examples_seen=0 epoch=0: 8.294288
2021-05-11 09:50:18: Loss after num_examples_seen=100 epoch=1: 8.281333
2021-05-11 09:50:32: Loss after num_examples_seen=200 epoch=2: 8.260510
2021-05-11 09:50:48: Loss after num_examples_seen=300 epoch=3: 8.217373
2021-05-11 09:51:02: Loss after num_examples_seen=400 epoch=4: 8.120548
2021-05-11 09:51:17: Loss after num_examples_seen=500 epoch=5: 7.796752
2021-05-11 09:51:31: Loss after num_examples_seen=600 epoch=6: 6.180859
2021-05-11 09:51:46: Loss after num_examples_seen=700 epoch=7: 5.855196
2021-05-11 09:52:01: Loss after num_examples_seen=800 epoch=8: 5.694309
2021-05-11 09:52:15: Loss after num_examples_seen=900 epoch=9: 5.587199


In [223]:
import theano as theano
import theano.tensor as T
import operator


In [224]:

def save_model_parameters_theano(outfile, model):
    U, V, W = model.U.get_value(), model.V.get_value(), model.W.get_value()
    np.savez(outfile, U=U, V=V, W=W)
    print ("Saved model parameters to %s." % outfile)
   
def load_model_parameters_theano(path, model):
    npzfile = np.load(path)
    U, V, W = npzfile["U"], npzfile["V"], npzfile["W"]
    model.hidden_dim = U.shape[0]
    model.word_dim = U.shape[1]
    model.U.set_value(U)
    model.V.set_value(V)
    model.W.set_value(W)
    print ("Loaded model parameters from %s. hidden_dim=%d word_dim=%d" % (path, U.shape[0], U.shape[1]))

class RNNTheano:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        # Theano: Created shared variables
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))      
        # We store the Theano graph here
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.ivector('x')
        y = T.ivector('y')
        def forward_prop_step(x_t, s_t_prev, U, V, W):
            s_t = T.tanh(U[:,x_t] + W.dot(s_t_prev))
            o_t = T.nnet.softmax(V.dot(s_t))
            return [o_t[0], s_t]
        [o,s], updates = theano.scan(
            forward_prop_step,
            sequences=x,
            outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
            non_sequences=[U, V, W],
            truncate_gradient=self.bptt_truncate,
            strict=True)
        
        prediction = T.argmax(o, axis=1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        
        # Gradients
        dU = T.grad(o_error, U)
        dV = T.grad(o_error, V)
        dW = T.grad(o_error, W)
        
        # Assign functions
        self.forward_propagation = theano.function([x], o)
        self.predict = theano.function([x], prediction)
        self.ce_error = theano.function([x, y], o_error)
        self.bptt = theano.function([x, y], [dU, dV, dW])
        
        # SGD
        learning_rate = T.scalar('learning_rate')
        self.sgd_step = theano.function([x,y,learning_rate], [], 
                      updates=[(self.U, self.U - learning_rate * dU),
                              (self.V, self.V - learning_rate * dV),
                              (self.W, self.W - learning_rate * dW)])
    
    def calculate_total_loss(self, X, Y):
        return np.sum([self.ce_error(x,y) for x,y in zip(X,Y)])
    
    def calculate_loss(self, X, Y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y) for y in Y])
        return self.calculate_total_loss(X,Y)/float(num_words)   


def gradient_check_theano(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter_T = operator.attrgetter(pname)(model)
        parameter = parameter_T.get_value()
        print ("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            parameter_T.set_value(parameter)
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            parameter_T.set_value(parameter)
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            parameter_T.set_value(parameter)
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print ("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print ("+h Loss: %f" % gradplus)
                print ("-h Loss: %f" % gradminus)
                print ("Estimated_gradient: %f" % estimated_gradient)
                print ("Backpropagation gradient: %f" % backprop_gradient)
                print ("Relative Error: %f" % relative_error)
                return 
            it.iternext()
        print ("Gradient check for parameter %s passed." % (pname))

In [225]:

np.random.seed(10)
model = RNNTheano(vocabulary_size,hidden_dim=50)
%timeit model.sgd_step(XTrain[10], yTrain[10], 0.005)

33.5 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [226]:
print("end")

end


In [231]:

def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    count =  0
    
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        print('count',count)
        count+=1
        next_word_probs = model.forward_propagation(new_sentence)[0]
        #next_word_probs=model.predict(new_sentence[-1])
        next_word_probs = next_word_probs.astype(float)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            print("in here too ")
            samples = np.random.multinomial(1, next_word_probs)#next_word_probs[-1]
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 10
senten_min_length = 7
"""
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    count =  0
    
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        print('count',count)
        count+=1

        next_word_probs=model.predict(new_sentence[])
        next_word_probs=np.asarray(next_word_probs).astype('float64')
        samples = np.random.multinomial(1, next_word_probs[0])#next_word_probs[-1]
        sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
        if len(new_sentence) > 100 or sampled_word == word_to_index[UNKNOWN_TOKEN]:
                return None
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    if len(new_sentence) < senten_min_length:
        return None    
    return sentence_str


"""
print("pre generation ")
 

pre generation 


In [232]:
 
print("Now generating\n") 
for i in range(num_sentences):
    print("i:",i)
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print (" ".join(sent))

    
print("\nend")


Now generating

i: 0
count 0
in here too 
count 1
in here too 
count 2
in here too 
count 3
in here too 
count 4
in here too 
count 5
in here too 
count 6
in here too 
count 7
in here too 
count 8
in here too 
count 9
in here too 
count 10
in here too 
count 11
in here too 
count 12
in here too 
count 13
in here too 
count 14
in here too 
count 15
in here too 
count 16
in here too 
count 17
in here too 
count 18
in here too 
count 19
in here too 
count 20
in here too 
count 21
in here too 
count 22
in here too 
count 23
in here too 
count 24
in here too 
count 25
in here too 
count 26
in here too 
count 27
in here too 
count 28
in here too 
count 29
in here too 
count 30
in here too 
count 31
in here too 
count 32
in here too 
count 33
in here too 
count 34
in here too 
count 35
in here too 
count 36
in here too 
count 37
in here too 
count 38
in here too 
count 39
in here too 
count 40
in here too 
count 41
in here too 
count 42
in here too 
count 43
in here too 
count 44
in here too 

in here too 
count 361
in here too 
count 362
in here too 
count 363
in here too 
count 364
in here too 
count 365
in here too 
count 366
in here too 
count 367
in here too 
count 368
in here too 
count 369
in here too 
count 370
in here too 
count 371
in here too 
count 372
in here too 
count 373
in here too 
count 374
in here too 
count 375
in here too 
count 376
in here too 
count 377
in here too 
count 378
in here too 
count 379
in here too 
count 380
in here too 
count 381
in here too 
count 382
in here too 
count 383
in here too 
count 384
in here too 
count 385
in here too 
count 386
in here too 
count 387
in here too 
count 388
in here too 
count 389
in here too 
count 390
in here too 
count 391
in here too 
count 392
in here too 
count 393
in here too 
count 394
in here too 
count 395
in here too 
count 396
in here too 
count 397
in here too 
count 398
in here too 
count 399
in here too 
count 400
in here too 
count 401
in here too 
count 402
in here too 
count 403
in here too

in here too 
count 718
in here too 
count 719
in here too 
count 720
in here too 
count 721
in here too 
count 722
in here too 
count 723
in here too 
count 724
in here too 
count 725
in here too 
count 726
in here too 
count 727
in here too 
count 728
in here too 
count 729
in here too 
count 730
in here too 
count 731
in here too 
count 732
in here too 
count 733
in here too 
count 734
in here too 
count 735
in here too 
count 736
in here too 
count 737
in here too 
count 738
in here too 
count 739
in here too 
count 740
in here too 
count 741
in here too 
count 742
in here too 
count 743
in here too 
count 744
in here too 
count 745
in here too 
count 746
in here too 
count 747
in here too 
count 748
in here too 
count 749
in here too 
count 750
in here too 
count 751
in here too 
count 752
in here too 
count 753
in here too 
count 754
in here too 
count 755
in here too 
count 756
in here too 
count 757
in here too 
count 758
in here too 
count 759
in here too 
count 760
in here too

in here too 
count 1071
in here too 
count 1072
in here too 
count 1073
in here too 
count 1074
in here too 
count 1075
in here too 
count 1076
in here too 
count 1077
in here too 
count 1078
in here too 
count 1079
in here too 
count 1080
in here too 
count 1081
in here too 
count 1082
in here too 
count 1083
in here too 
count 1084
in here too 
count 1085
in here too 
count 1086
in here too 
count 1087
in here too 
count 1088
in here too 
count 1089
in here too 
count 1090
in here too 
count 1091
in here too 
count 1092
in here too 
count 1093
in here too 
count 1094
in here too 
count 1095
in here too 
count 1096
in here too 
count 1097
in here too 
count 1098
in here too 
count 1099
in here too 
count 1100
in here too 
count 1101
in here too 
count 1102
in here too 
count 1103
in here too 
count 1104
in here too 
count 1105
in here too 
count 1106
in here too 
count 1107
in here too 
count 1108
in here too 
count 1109
in here too 
count 1110
in here too 
count 1111
in here too 
cou

in here too 
count 1413
in here too 
count 1414
in here too 
count 1415
in here too 
count 1416
in here too 
count 1417
in here too 
count 1418
in here too 
count 1419
in here too 
count 1420
in here too 
count 1421
in here too 
count 1422
in here too 
count 1423
in here too 
count 1424
in here too 
count 1425
in here too 
count 1426
in here too 
count 1427
in here too 
count 1428
in here too 
count 1429
in here too 
count 1430
in here too 
count 1431
in here too 
count 1432
in here too 
count 1433
in here too 
count 1434
in here too 
count 1435
in here too 
count 1436
in here too 
count 1437
in here too 
count 1438
in here too 
count 1439
in here too 
count 1440
in here too 
count 1441
in here too 
count 1442
in here too 
count 1443
in here too 
count 1444
in here too 
count 1445
in here too 
count 1446
in here too 
count 1447
in here too 
count 1448
in here too 
count 1449
in here too 
count 1450
in here too 
count 1451
in here too 
count 1452
in here too 
count 1453
in here too 
cou

...

in here too 
count 2439
in here too 
count 2440
in here too 
count 2441
in here too 
count 2442
in here too 
count 2443
in here too 
count 2444
in here too 
count 2445
in here too 
count 2446
in here too 
count 2447
in here too 
count 2448
in here too 
count 2449
in here too 
count 2450
in here too 
count 2451
in here too 
count 2452
in here too 
count 2453
in here too 
count 2454
in here too 
count 2455
in here too 
count 2456
in here too 
count 2457
in here too 
count 2458
in here too 
count 2459
in here too 
count 2460
in here too 
count 2461
in here too 
count 2462
in here too 
count 2463
in here too 
count 2464
in here too 
count 2465
in here too 
count 2466
in here too 
count 2467
in here too 
count 2468
in here too 
count 2469
in here too 
count 2470
in here too 
count 2471
in here too 
count 2472
in here too 
count 2473
in here too 
count 2474
in here too 
count 2475
in here too 
count 2476
in here too 
count 2477
in here too 
count 2478
in here too 
count 2479
in here too 
cou

in here too 
count 2781
in here too 
count 2782
in here too 
count 2783
in here too 
count 2784
in here too 
count 2785
in here too 
count 2786
in here too 
count 2787
in here too 
count 2788
in here too 
count 2789
in here too 
count 2790
in here too 
count 2791
in here too 
count 2792
in here too 
count 2793
in here too 
count 2794
in here too 
count 2795
in here too 
count 2796
in here too 
count 2797
in here too 
count 2798
in here too 
count 2799
in here too 
count 2800
in here too 
count 2801
in here too 
count 2802
in here too 
count 2803
in here too 
count 2804
in here too 
count 2805
in here too 
count 2806
in here too 
count 2807
in here too 
count 2808
in here too 
count 2809
in here too 
count 2810
in here too 
count 2811
in here too 
count 2812
in here too 
count 2813
in here too 
count 2814
in here too 
count 2815
in here too 
count 2816
in here too 
count 2817
in here too 
count 2818
in here too 
count 2819
in here too 
count 2820
in here too 
count 2821
in here too 
cou

in here too 
count 3123
in here too 
count 3124
in here too 
count 3125
in here too 
count 3126
in here too 
count 3127
in here too 
count 3128
in here too 
count 3129
in here too 
count 3130
in here too 
count 3131
in here too 
count 3132
in here too 
count 3133
in here too 
count 3134
in here too 
count 3135
in here too 
count 3136
in here too 
count 3137
in here too 
count 3138
in here too 
count 3139
in here too 
count 3140
in here too 
count 3141
in here too 
count 3142
in here too 
count 3143
in here too 
count 3144
in here too 
count 3145
in here too 
count 3146
in here too 
count 3147
in here too 
count 3148
in here too 
count 3149
in here too 
count 3150
in here too 
count 3151
in here too 
count 3152
in here too 
count 3153
in here too 
count 3154
in here too 
count 3155
in here too 
count 3156
in here too 
count 3157
in here too 
count 3158
in here too 
count 3159
in here too 
count 3160
in here too 
count 3161
in here too 
count 3162
in here too 
count 3163
in here too 
cou

in here too 
count 3464
in here too 
count 3465
in here too 
count 3466
in here too 
count 3467
in here too 
count 3468
in here too 
count 3469
in here too 
count 3470
in here too 
count 3471
in here too 
count 3472
in here too 
count 3473
in here too 
count 3474
in here too 
count 3475
in here too 
count 3476
in here too 
count 3477
in here too 
count 3478
in here too 
count 3479
in here too 
count 3480
in here too 
count 3481
in here too 
count 3482
in here too 
count 3483
in here too 
count 3484
in here too 
count 3485
in here too 
count 3486
in here too 
count 3487
in here too 
count 3488
in here too 
count 3489
in here too 
count 3490
in here too 
count 3491
in here too 
count 3492
in here too 
count 3493
in here too 
count 3494
in here too 
count 3495
in here too 
count 3496
in here too 
count 3497
in here too 
count 3498
in here too 
count 3499
in here too 
count 3500
in here too 
count 3501
in here too 
count 3502
in here too 
count 3503
in here too 
count 3504
in here too 
cou

KeyboardInterrupt: 